In [68]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

In [69]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [70]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [71]:
%%capture pwd
!pwd

In [72]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-does-the-housing-market-affect-wealth-inequality', 300, 500)

# Fig 1

In [73]:
df = pd.read_csv("raw/fig1.csv")
df['y1']/=100
df['y2']/=100
df['y3']/=100
df['y4']/=100

In [74]:
f = "fig1_home_ownership"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,x1,y1,x2,y2,x3,y3,x4,y4
0,1961.221596,0.303993,1961.221596,0.287943,1961.221596,0.270147,1961.221596,0.253399
1,1961.669214,0.298410,1961.669214,0.287943,1961.702786,0.274544,1961.669214,0.258284
2,1962.116832,0.295619,1962.060880,0.291199,1962.135483,0.277707,1962.038499,0.265053
3,1962.564451,0.299108,1962.340641,0.307482,1962.788260,0.292130,1962.452546,0.254446
4,1963.012069,0.303295,1962.564451,0.322137,1963.235878,0.292130,1962.900164,0.243280


In [75]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "x1:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            # title="⬅ historical    forecast ➡",
            title='',
            titleY=-295,
            titleX=216,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=5,
            orient="bottom",
            labelAngle=0,
            format='.0f'
        ),
        # scale=alt.Scale(domain=[2019.9, 2026],nice=False),
    ),
    y=alt.Y(
        "y1:Q",
        sort=[],
        axis=alt.Axis(
            # grid=False,
            title="",
            titleX=-5,
            titleY=-5,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            format='p'
        ),
        # scale=alt.Scale(domain=[-1, 6]),
    )
)
line1 = base.mark_line(color=colors["eco-dot"],interpolate='basis').encode(
)
base=base.encode(x='x2:Q',y='y2:Q')
line2 = base.mark_line(color=colors["eco-mid-blue"],interpolate='basis').encode(
)
base=base.encode(x='x3:Q',y='y3:Q')
line3 = base.mark_line(color=colors["eco-light-blue"],interpolate='basis').encode(
)
base=base.encode(x='x4:Q',y='y4:Q')
line4 = base.mark_line(color=colors["eco-turquiose"],interpolate='basis').encode(
)
label1=alt.Chart(pd.DataFrame([{'x':2019.2,'y':.53,'t':'All ages'}])).mark_text(color=colors['eco-dot'],align='left').encode(x='x:Q',y='y:Q',text='t:N')
label2=alt.Chart(pd.DataFrame([{'x':2019.2,'y':.375,'t':'30-34 year olds'}])).mark_text(color=colors['eco-mid-blue'],align='left').encode(x='x:Q',y='y:Q',text='t:N')
label3=alt.Chart(pd.DataFrame([{'x':2019.2,'y':.285,'t':'25-34 year olds'}])).mark_text(color=colors['eco-light-blue'],align='left').encode(x='x:Q',y='y:Q',text='t:N')
label4=alt.Chart(pd.DataFrame([{'x':2019.2,'y':.195,'t':'25-29 year olds'}])).mark_text(color=colors['eco-turquiose'],align='left').encode(x='x:Q',y='y:Q',text='t:N')
layer1 = (
    ((line1+line2+line3+line4+label1+label2+label3+label4).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png")
layer1.save("visualisation/" + f + ".svg")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)